In [0]:
# Part 1.1: Row Counts
This query counts the number of rows in each bronze table: `device_messages` and `rapid_step_tests`. It helps you understand the data volume before analysis and transformation.

In [0]:
# Part 1.2: Top Devices by Message Count
This query shows the top 10 devices by number of messages received, helping identify the most active devices in the dataset.

In [0]:
# Part 1.3: Clean and Cast Distance
This query creates a temporary view that cleans the `distance` field, converting values like "1cm" to numeric centimeters. It also standardizes timestamps and selects relevant columns for analysis.

In [0]:
# Part 1.4: Per-Device Distance Stats
This query calculates the average, minimum, and maximum cleaned distance per device, showing sensor performance and variability.

In [0]:
# Part 1.5: Explode Step Points
This query explodes the `step_points` array in each rapid step test, so each row represents a single step interval. It then summarizes step timing stats per device and customer.

In [0]:
# Part 1.6: Time-Window Join
This query joins sensor readings to rapid step tests by device and time window, aggregating readings that occurred during each test. These aggregates will be reused as ML features later.

In [0]:
# Part 2.1: Load Tables
This cell loads the bronze tables as Spark DataFrames and prints their schemas, preparing for further analysis and transformation.

In [0]:
# Part 2.2: Clean Distance to Numeric
This cell cleans the `distance` field in the device messages, converting values like "1cm" to numeric centimeters and standardizing timestamps for analysis.

In [0]:
# Part 2.3: Per-Device Descriptive Stats
This cell computes per-device statistics (count, average, min, max) for cleaned distance values, helping identify sensor performance and outliers.

In [0]:
# Part 2.4: Explode Step Points and Stats
This cell explodes the `step_points` array in rapid step tests, then summarizes step timing stats per test. This enables richer analysis of step intervals.

In [0]:
# Part 2.5: Windowed Join for Feature Prep
This cell joins device messages to rapid step tests by device and time window, aggregating sensor readings within each test. These features will be reused for ML.

In [0]:
# Part 2.6: Quick Visual Check
This cell samples one device and plots its cleaned distance readings over time, providing a quick visual check of sensor data quality and trends.

In [0]:
%sql
SELECT 'device_messages_raw' AS table_name, COUNT(*) AS rows FROM workspace.bronze.device_messages

UNION ALL

SELECT 'rapid_step_tests_raw', COUNT(*) FROM workspace.bronze.rapid_step_tests;


In [0]:
%sql
SELECT device_id, COUNT(*) AS messages
FROM workspace.bronze.device_messages
GROUP BY device_id
ORDER BY messages DESC
LIMIT 10;

In [0]:
%sql
-- Create a cleaned view for convenience during lab (session-scoped)

CREATE OR REPLACE TEMP VIEW device_messages_clean AS
SELECT
  device_id,
  TRY_CAST(REGEXP_REPLACE(CAST(distance AS STRING), '[^0-9.]', '') AS DOUBLE) AS distance_cm,
  CAST(timestamp AS BIGINT) AS ts_ms,
  CAST(date AS BIGINT)       AS date_ms,
  message_origin,
  sensor_type,
  message
FROM workspace.bronze.device_messages;

-- Check the cleaning
SELECT * FROM device_messages_clean LIMIT 20;

In [0]:
%sql
SELECT device_id,

       AVG(distance_cm) AS avg_cm,

       MIN(distance_cm) AS min_cm,

       MAX(distance_cm) AS max_cm

FROM device_messages_clean

GROUP BY device_id

ORDER BY avg_cm DESC

LIMIT 10;

FROM workspace.bronze.rapid_step_tests;



In [0]:
%sql
-- One row per step interval

WITH exploded AS (
  SELECT
    customer,
    device_id,
    start_time,
    posexplode(step_points) AS (step_index, step_ms)
  FROM workspace.bronze.rapid_step_tests
)
SELECT customer, device_id,
       COUNT(*) AS steps,
       AVG(step_ms) AS avg_step_ms,
       STDDEV(step_ms) AS sd_step_ms
FROM exploded
GROUP BY customer, device_id
ORDER BY steps DESC
LIMIT 10;

In [0]:
%sql
-- Join messages that occurred during each test window for the same device

WITH tests AS (

  SELECT customer, device_id, start_time, stop_time, test_time, total_steps

  FROM workspace.bronze.rapid_step_tests

),

msgs AS (

  SELECT device_id, ts_ms, distance_cm

  FROM device_messages_clean

  WHERE distance_cm IS NOT NULL

)

SELECT

  t.customer, t.device_id, t.start_time, t.stop_time,

  COUNT(m.ts_ms)                   AS readings_in_window,

  AVG(m.distance_cm)               AS avg_cm_in_window,

  MIN(m.distance_cm)               AS min_cm_in_window,

  MAX(m.distance_cm)               AS max_cm_in_window

FROM tests t

JOIN msgs  m

  ON m.device_id = t.device_id

 AND m.ts_ms BETWEEN t.start_time AND t.stop_time

GROUP BY t.customer, t.device_id, t.start_time, t.stop_time

ORDER BY readings_in_window DESC

LIMIT 20;


In [0]:
# Spark DataFrames from the catalog

dm = spark.table("workspace.bronze.device_messages")

rt = spark.table("workspace.bronze.rapid_step_tests")

dm.printSchema()

rt.printSchema()


In [0]:
from pyspark.sql import functions as F

dm_clean = (dm

    .withColumn("distance_str", F.col("distance").cast("string"))

    .withColumn("distance_cm", F.regexp_replace("distance_str", "[^0-9.]", "").cast("double"))

    .withColumn("ts_ms", F.col("timestamp").cast("bigint"))

    .withColumn("date_ms", F.col("date").cast("bigint"))

)

display(dm_clean.limit(10))


In [0]:
dm_stats = (dm_clean

    .groupBy("device_id")

    .agg(F.count("*").alias("n"),

         F.avg("distance_cm").alias("avg_cm"),

         F.min("distance_cm").alias("min_cm"),

         F.max("distance_cm").alias("max_cm"))

    .orderBy(F.desc("n"))

)

display(dm_stats.limit(20))


In [0]:
rt_exploded = (rt

    .select("customer","device_id","start_time","stop_time","test_time","total_steps", F.posexplode("step_points").alias("step_index","step_ms"))

)

display(rt_exploded.limit(20))

# Step timing stats per test

step_stats = (rt_exploded

    .groupBy("customer","device_id","start_time","stop_time")

    .agg(F.count("*").alias("steps"),

         F.avg("step_ms").alias("avg_step_ms"),

         F.stddev("step_ms").alias("sd_step_ms"))

    .orderBy(F.desc("steps"))

)

display(step_stats.limit(20))


In [0]:
# Filter to valid numeric distances

msgs = dm_clean.select("device_id", "ts_ms", "distance_cm").where(F.col("distance_cm").isNotNull())

tests = rt.select("customer","device_id","start_time","stop_time","test_time","total_steps")

joined = (tests.alias("t")
    .join(msgs.alias("m"),
          (F.col("m.device_id")==F.col("t.device_id")) &
          (F.col("m.ts_ms").between(F.col("t.start_time"), F.col("t.stop_time"))),
          how="inner")
)

features = (joined
    .groupBy("t.customer","t.device_id","t.start_time","t.stop_time","t.test_time","t.total_steps")
    .agg(F.count("m.ts_ms").alias("readings_in_window"),
         F.avg("m.distance_cm").alias("avg_cm_in_window"),
         F.min("m.distance_cm").alias("min_cm_in_window"),
         F.max("m.distance_cm").alias("max_cm_in_window"),
         F.var_pop("m.distance_cm").alias("var_cm_in_window"))
    .orderBy(F.desc("readings_in_window"))
)
display(features.limit(20))


In [0]:
# Small sample for a simple line plot of distances over time for one device

import pandas as pd
import matplotlib.pyplot as plt

sample_device = features.select("device_id").first()["device_id"]

pdf = (dm_clean
       .filter(F.col("device_id") == sample_device)
       .orderBy("ts_ms")
       .limit(1000)
       .select("ts_ms","distance_cm")
       .toPandas())

plt.figure()
plt.plot(pdf["ts_ms"], pdf["distance_cm"])
plt.title(f"Distance over time (device {sample_device})")
plt.xlabel("timestamp (ms)")
plt.ylabel("distance (cm)")
plt.show()


In [0]:
# Lab Reflection
The SQL and Python steps for cleaning and joining the STEDI data were straightforward, especially with clear schema and column names. The time window join logic was a bit confusing at first, but seeing it in both SQL and Python helped clarify how to align sensor readings to test intervals. One ethics risk I noticed is the potential for misinterpreting sensor data if cleaning steps are not transparent, this could lead to misleading results or unfair model outcomes. Responsible documentation and validation are essential.